In [1]:
import requests
from bs4 import BeautifulSoup
import subprocess
import re
import os
import stanfordnlp
import pandas as pd

from ProcessArticle import ProcessArticle

from WikiXmlHandler import WikiXmlHandler
import xml.sax
import mwparserfromhell

from timeit import default_timer as timer

from multiprocessing import Pool 
from multiprocessing import Lock
import tqdm 

# List of lists to single list
from itertools import chain

# Sending keyword arguments in map
from functools import partial

import gc

In [2]:
infoboxes = pd.read_csv("./saved/infobox-list.csv")["infobox template name"].values.tolist()
folder = "./data"
files = [folder + "/" + x for x in os.listdir(folder)]
file = files[0] # process only one file for now

out_file = "./saved/out_nlp.csv"

nlp = stanfordnlp.Pipeline(models_dir="./stanfordnlp_resources/", use_gpu=False);

func = ProcessArticle(infoboxes, nlp)

Use device: cpu
---
Loading: tokenize
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings:
{'model_path': './stanfordnlp_resources/en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': './stanfordnlp_resources/en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': '

In [3]:
%%time

handler = WikiXmlHandler(out_file, func, 1000)
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

print("Started processing: {}".format(file))
for line in subprocess.Popen(["bzcat"],
                        stdin=open(file),
                        stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

# making sure that everything was written to the file
handler.write_df()

th dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
..\aten\src\ATen\native\LegacyDefinitions.cpp:14: UserWarning: masked_fill_ received a mask with dtype torch.